# Бутстрэп без лишних логов
Пример оценивания среднего значения.

In [1]:
import numpy as np
import pandas as pd
from policyscope.bootstrap import cluster_bootstrap_ci

rng = np.random.default_rng(0)
df = pd.DataFrame({
    'user_id': np.arange(100),
    'accept': rng.binomial(1, 0.3, size=100),
})

def estimator(sub):
    return sub['accept'].mean()

theta, low, high = cluster_bootstrap_ci(df, estimator, cluster_col='user_id', n_boot=100, alpha=0.05, rng_seed=1)
print(theta, low, high)


0.37 0.30306451612903224 0.4595161290322579
